In [16]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.special import gamma, kv
from numpy import matlib

import torch
import mgplvm as mgp
import pickle
import time
from sklearn.decomposition import FactorAnalysis
from sklearn.linear_model import LinearRegression, Ridge
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter1d
plt.rcParams['font.size'] = 10
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
np.random.seed(0)
torch.manual_seed(0)
device = mgp.utils.get_device() # use GPU if available, otherwise CPU
print(device)

from scipy.stats import poisson
from synthetic_data import *

import sys
import io

from sklearn.model_selection import train_test_split

cuda


In [17]:
# mod = pickle.load(open('models/50k_kernel_0_seed_0model.pickled', 'rb'))
mod = pickle.load(open('models/50k_r1_kernel_0_seed_0model.pickled', 'rb'))


In [18]:
data = pickle.load(open('data/Doherty_example.pickled', 'rb')) # load example data
binsize = 25 # binsize in ms
timepoints = np.arange(0, 50000) #subsample ~40 seconds of data so things will run somewhat quicker
print(data['Y'].shape)
fit_data = {'Y': data['Y'][..., timepoints], 'locs': data['locs'][timepoints, :], 'targets': data['targets'][timepoints, :], 'binsize': binsize}
# fit_data = {'Y': data['Y'], 'locs': data['locs'], 'targets': data['targets'], 'binsize': binsize}
Y = fit_data['Y'] # these are the actual recordings and is the input to our model
targets = fit_data['targets'] # these are the target locations
locs = fit_data['locs'] # these are the hand positions
# print(Y.shape)
Y = Y[:, np.mean(Y,axis = (0, 2))/0.025 > 2, :] #subsample highly active neurons so things will run a bit quicker
# print(Y.shape)
ntrials, n, T = Y.shape # Y should have shape: [number of trials (here 1) x neurons x time points]
data = torch.tensor(Y).to(device) # put the data on our GPU/CPU
ts = np.arange(Y.shape[-1]) #much easier to work in units of time bins here
fit_ts = torch.tensor(ts)[None, None, :].to(device) # put our time points on GPU/CPU

# finally let's just identify bins where the target changes
deltas = np.concatenate([np.zeros(1), np.sum(np.abs(targets[1:, :] - targets[:-1, :]), axis = 1)])
switches = np.where(deltas > 1e-5)[0] # change occurs during time bin s
dswitches = np.concatenate([np.ones(1)*10, switches[1:] - switches[:-1]]) # when the target changes during a bin there will be two discontinuities
inds = np.zeros(len(switches)).astype(bool)
inds[dswitches > 1.5] = 1 # index of the bin where the target changes or the first bin with a new target
switches = switches[inds]

# print(np.mean(Y, axis = (0, 2)))

### set some parameters for fitting ###
ell0 = 200/binsize # initial timescale (in bins) for each dimension. This could be the ~timescale of the behavior of interest (otherwise a few hundred ms is a reasonable default)
rho = 2 # sets the intial scale of each latent (s_d in Jensen & Kao). rho=1 is a natural choice with Gaussian noise; less obvious with non-Gaussian noise but rho=1-5 works well empirically.
max_steps = 1001 # number of training iterations
# n_mc = 5 # number of monte carlo samples per iteration
print_every = 100 # how often we print training progress
d_fit = 20 # lets fit up to 1


### construct the actual model ###
ntrials, n, T = Y.shape # Y should have shape: [number of trials (here 1) x neurons x time points]

ts = np.arange(Y.shape[-1])*fit_data['binsize'] # measured in ms
cs = CubicSpline(ts, locs) # fit cubic spline to behavior
vels = cs(ts, 1) # velocity (first derivative)

(1, 200, 70482)


In [19]:
from sklearn.model_selection import train_test_split
### finally let's do a simple decoding analysis ###
torch.cuda.empty_cache() # clear GPU memory
print('running decoding analysis')
Ypreds = [] # decode from the inferred firing rates (this is a non-linear decoder from latents)
query = mod.lat_dist.lat_mu.detach().transpose(-1, -2).to(device)  # (ntrial, d_fit, T)
for i in range(100): # loop over mc samples to avoid memory issues
    Ypred = mod.svgp.sample(query, n_mc=10, noise=False) # OG n_mc = 100
    Ypred = Ypred.detach().mean(0).cpu().numpy()  # (ntrial x n x T)
    Ypreds.append(Ypred)
Ypred = np.mean(np.array(Ypreds), axis = (0,1)).T # T x n

running decoding analysis


In [20]:
# delays = np.linspace(0, 250, 100) # consider different behavioral delays
# performance = np.zeros((len(delays), 2)) # model performance
# for idelay, delay in enumerate(delays):
#     vels = cs(ts+delay, 1) # velocity at time+delay
#     for itest, Ytest in enumerate([Ypred]): # bGPFA
#         regs = [Ridge(alpha=1e-3).fit(Ytest[::2, :], vels[::2, i]) for i in range(2)] # fit x and y vel on half the data
#         scores = [regs[i].score(Ytest[1::2, :], vels[1::2, i]) for i in range(2)] # score x and y vel on the other half
#         # regs = [Ridge(alpha=1e-3).fit(Ytest[2400:, :], vels[2400:, i]) for i in range(2)] # fit x and y vel on half the data
#         # scores = [regs[i].score(Ytest[:2400, :], vels[:2400, i]) for i in range(2)] # score x and y vel on the other half
#         # regs = [Ridge(alpha=1e-3).fit(Ytest, vels[:, i]) for i in range(2)]
#         # scores = [regs[i].score(Ytest, vels[:, i]) for i in range(2)]
#         performance[idelay, itest] = np.mean(scores) # save performance
# print('plotting decoding')
# plt.figure()
# plt.plot(delays, performance[:, 0], 'k-')
# print(max(performance[:, 0]))
# plt.axvline(delays[np.argmax(performance[:, 0])], color = 'b', ls = '--')
# plt.xlim(delays[0], delays[-1])
# plt.xlabel('delay (ms)')
# plt.ylabel('kinematic decoding')
# plt.show()

folds = 10
delay = 100
performance = np.zeros(folds)
for fold in range(folds):
    # train, test = train_test_split(np.arange(Y.shape[-1]), test_size=0.1, random_state=fold)
    test_size = int(Y.shape[-1] / folds)
    test_start = fold * test_size
    test_end = test_start + test_size
    test = np.arange(test_start, test_end)
    train = np.concatenate([np.arange(0, test_start), np.arange(test_end, Y.shape[-1])])
    print(train , test)
# delays = np.linspace(0, 250, 100) # consider different behavioral delays
# performance = np.zeros((len(delays), 2)) # model performance
# for idelay, delay in enumerate(delays):
    vels = cs(ts+delay, 1) # velocity at time+delay
    for itest, Ytest in enumerate([Ypred]): # bGPFA
        regs = [Ridge(alpha=1e-3).fit(Ytest[train], vels[train]) for i in range(2)] # fit x and y vel on half the data
        scores = [regs[i].score(Ytest[test], vels[test]) for i in range(2)] # score x and y vel on the other half
        # regs = [Ridge(alpha=1e-3).fit(Ytest[2400:, :], vels[2400:, i]) for i in range(2)] # fit x and y vel on half the data
        # scores = [regs[i].score(Ytest[:2400, :], vels[:2400, i]) for i in range(2)] # score x and y vel on the other half
        # regs = [Ridge(alpha=1e-3).fit(Ytest, vels[:, i]) for i in range(2)]
        # scores = [regs[i].score(Ytest, vels[:, i]) for i in range(2)]
        performance[fold] = np.mean(scores) # save performance
print('plotting decoding')
print(np.mean(performance))
print(np.std(performance))

[ 5000  5001  5002 ... 49997 49998 49999] [   0    1    2 ... 4997 4998 4999]
[    0     1     2 ... 49997 49998 49999] [5000 5001 5002 ... 9997 9998 9999]


[    0     1     2 ... 49997 49998 49999] [10000 10001 10002 ... 14997 14998 14999]
[    0     1     2 ... 49997 49998 49999] [15000 15001 15002 ... 19997 19998 19999]
[    0     1     2 ... 49997 49998 49999] [20000 20001 20002 ... 24997 24998 24999]
[    0     1     2 ... 49997 49998 49999] [25000 25001 25002 ... 29997 29998 29999]
[    0     1     2 ... 49997 49998 49999] [30000 30001 30002 ... 34997 34998 34999]
[    0     1     2 ... 49997 49998 49999] [35000 35001 35002 ... 39997 39998 39999]
[    0     1     2 ... 49997 49998 49999] [40000 40001 40002 ... 44997 44998 44999]
[    0     1     2 ... 44997 44998 44999] [45000 45001 45002 ... 49997 49998 49999]
plotting decoding
0.7399021437201105
0.025524792760091213
